# Orbit determination example
This notebook does the following:
* Download an orbit first guess from SpaceTrack
* Download laser ranging data
* Feed the data to Orekit
* Estimate the orbit
* Propagate and compare the orbit to the first guess

First, some parameters need to be defined for the orbit determination:
* Satellite ID in NORAD and COSPAR format
* Spacecraft mass: important for the drag term
* Measurement weights: used to weight certain measurements more than others during the orbit estimation. Here, we only have range measurements and we do not know the confidence associated to these measurements, so all weights are identical
* OD date: date at which the orbit will be estimated. 
* Data collection duration: for example, if equals 2 days, the laser data from the 2 days before the OD date will be used to estimate the orbit. This value is an important trade-off for the quality of the orbit determination:
    * The longer the duration, the more ranging data is available, which can increase the quality of the estimation
    * The longer the duration, the longer the orbit must be propagated, and the higher the covariance because of the orbit perturbations such as the gravity field, drag, Sun, Moon, etc.

In [1]:
# Constants
c = 299792458 # m/s

# Parameters
noradId = 42829
cosparId = '1704205' 
mass = 20.0 # kg
weight_range = 1 # Will be normalized later (i.e divided by the number of observations)

from datetime import datetime
odDate = datetime(2018, 7, 14) # Beginning of the orbit determination
collectionDuration = 2.0 # days
from datetime import timedelta
startCollectionDate = odDate + timedelta(days=-collectionDuration)

Initializing Orekit and JVM

In [2]:
import orekit
orekit.initVM()
from orekit.pyhelpers import setup_orekit_curdir
setup_orekit_curdir()

Import station data from file

In [4]:
import crdUtils
stationData = crdUtils.parseCrdFile('SLRF2014_POS+VEL_2030.0_180504.snx')

The following sets up accounts for SpaceTrack (for orbit data) and the EDC API (for laser ranging data).
* A SpaceTrack account is required, it can be created for free at: https://www.space-track.org/auth/createAccount
* An EDC account is required, it can be created for free at: https://edc.dgfi.tum.de/en/register/

In [2]:
# Space-Track
identity_st = 'clement@jonglez.space' # Email address for Space-Track
import getpass
print('Enter SpaceTrack password for account {}'.format(identity_st))
password_st = getpass.getpass()
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
st = SpaceTrackClient(identity=identity_st, password=password_st)

# EDC API
username_edc = 'jonglez' # Username for EDC API
print('Enter EDC API password for account {}'.format(username_edc))
password_edc = getpass.getpass() # You will get prompted for your password
url = 'https://edc.dgfi.tum.de/api/v1/'

Enter SpaceTrack password for account clement@jonglez.space
········
Enter EDC API password for account jonglez
········


The orbit determination needs a first guess. For this, we use Two-Line Elements. Retrieving the latest TLE prior to the beginning of the orbit determination. It is important to have a "fresh" TLE, because the newer the TLE, the better the orbit estimation.

In [3]:
rawTle = st.tle(norad_cat_id=noradId, epoch='<{}'.format(odDate), orderby='epoch desc', limit=1, format='tle')
tleLine1 = rawTle.split('\n')[0]
tleLine2 = rawTle.split('\n')[1]
print(tleLine1)
print(tleLine2)

1 42829U 17042E   18194.87572169 +.00000240 +00000-0 +29796-4 0  9990
2 42829 097.5943 096.2213 0015726 108.7371 251.5556 14.90835887054309


Looking for laser ranging data prior to the OD date.

The API only allows to look for data using the date formats 2018-07-1%, 2018-07-14% or 2018-07-14 0% for example. As a consequence, the search must be split into several days. The results are then sorted, and the observations which are outside of the date range are deleted.

In [4]:
import requests
import json
search_args = {}
search_args['username'] = username_edc
search_args['password'] = password_edc
search_args['action'] = 'data-query'
search_args['data_type'] = 'NPT' # Normal pointing data
search_args['satellite'] = cosparId

station_args = {}
station_args['username'] = username_edc
station_args['password'] = password_edc
station_args['action'] = 'station-info'

observationList = []
stationLat = []
stationLon = []

for i in range(int(collectionDuration) + 1): # Making a request for each day
    endDate = startCollectionDate + timedelta(days=i)
    search_args['end_data_date'] = '{:%Y-%m-%d}%'.format(endDate)
    
    search_response = requests.post(url, data=search_args)

    if search_response.status_code == 200:
        search_data = json.loads(search_response.text)
        
        for observation in search_data:         
            startDataDate = datetime.strptime(observation['start_data_date'], '%Y-%m-%d %H:%M:%S')
            endDataDate = datetime.strptime(observation['end_data_date'], '%Y-%m-%d %H:%M:%S')
            
            if (startDataDate > startCollectionDate) and (endDataDate < odDate): # Only taking the values within the date range
                
                station_args['station_id'] = observation['station'][0:4]
                station_response = requests.post(url, data=station_args)

                if station_response.status_code == 200:
                    station_data = json.loads(station_response.text)
                    
                    stationLat.append(station_data['latitude'])
                    stationLon.append(station_data['longitude'])                        
                    observationList.append(observation['id']) 

                    print('Observation Id: {}  -  Station: {}  -  Date: {}'.format(observation['id'], station_data['code'], observation['end_data_date']))
                else:
                    print(station_response.status_code)
                    print(station_response.text)
                    
            
    else:
        print(search_response.status_code)
        print(search_response.text)

Observation Id: 1693874  -  Station: HERL  -  Date: 2018-07-12 00:29:32
Observation Id: 1693915  -  Station: GODL  -  Date: 2018-07-12 03:46:22
Observation Id: 1694049  -  Station: THTL  -  Date: 2018-07-12 08:52:16
Observation Id: 1696090  -  Station: STL3  -  Date: 2018-07-12 12:11:57
Observation Id: 1694182  -  Station: YARL  -  Date: 2018-07-12 15:19:28
Observation Id: 1694184  -  Station: YARL  -  Date: 2018-07-12 15:23:46
Observation Id: 1694509  -  Station: GRZL  -  Date: 2018-07-12 21:31:25
Observation Id: 1694268  -  Station: MATM  -  Date: 2018-07-12 21:29:24
Observation Id: 1694421  -  Station: GRZL  -  Date: 2018-07-12 23:07:45
Observation Id: 1694485  -  Station: GODL  -  Date: 2018-07-13 03:59:00
Observation Id: 1694575  -  Station: GODL  -  Date: 2018-07-13 05:33:47
Observation Id: 1694705  -  Station: THTL  -  Date: 2018-07-13 09:03:01
Observation Id: 1694604  -  Station: MATM  -  Date: 2018-07-13 09:54:35
Observation Id: 1696456  -  Station: SOSW  -  Date: 2018-07-13 1

Downloading the list of observations.

In [5]:
dl_args = {}
dl_args['username'] = username_edc
dl_args['password'] = password_edc
dl_args['action'] = 'data-download'
dl_args['data_type'] = 'NPT'

rangeArray = []

for obsId in observationList:
    dl_args['id'] = obsId
    dl_response = requests.post(url, data=dl_args)

    if dl_response.status_code == 200:
        """ convert json string in python list """
        data = json.loads(dl_response.text)
        #print(json.dumps(data, indent=4, sort_keys=True))
        
        currentLine = ''
        i = 0
        n = len(data)

        while (not currentLine.lower().startswith('h4')) and i < n: # Reading lines until the H4 header
            currentLine = data[i]
            i += 1
            
        #print(currentLine)

        lineData = currentLine.split() # Reading day in H4 header
        y = int(lineData[2])
        m = int(lineData[3])
        d = int(lineData[4])
        from datetime import datetime
        from datetime import timedelta
        measurementDay = datetime(y, m, d)

        while (not currentLine.startswith('11')) and i < n: # Reading lines until the start of normal point data
            currentLine = data[i]
            i += 1

        while currentLine.startswith('11') and i < n: # Reading until the end of normal point data
            lineData = currentLine.split()
            timeOfDay = float(lineData[1])
            timeOfFlight = float(lineData[2])
            timestampType = int(lineData[4])

            r = c * timeOfFlight / 2

            if timestampType == 1:
                transmitTime = measurementDay + timedelta(seconds=(timeOfDay - timeOfFlight/2))
            else:
                transmitTime = measurementDay + timedelta(seconds=timeOfDay)

            bounceTime = transmitTime + timedelta(seconds=timeOfFlight/2)
            receiveTime = bounceTime + timedelta(seconds=timeOfFlight/2)

            print('Transmit time: {}, receive time: {}'.format(transmitTime, receiveTime))
            print('Time of flight: {} milliseconds, satellite range: {} kilometers'.format(timeOfFlight*1000, r/1000))
            print('')

            currentLine = data[i]
            i += 1
            
    else:
        print(dl_response.status_code)
        print(dl_response.text)

Transmit time: 2018-07-12 00:28:09.917022, receive time: 2018-07-12 00:28:09.924548
Time of flight: 7.525834575 milliseconds, satellite range: 1128.0942228703177 kilometers

Transmit time: 2018-07-12 00:28:23.406022, receive time: 2018-07-12 00:28:23.413488
Time of flight: 7.465774903 milliseconds, satellite range: 1119.0915045225408 kilometers

Transmit time: 2018-07-12 00:28:37.050022, receive time: 2018-07-12 00:28:37.057486
Time of flight: 7.464117337999999 milliseconds, satellite range: 1118.8430417797183 kilometers

Transmit time: 2018-07-12 00:28:50.784022, receive time: 2018-07-12 00:28:50.791544
Time of flight: 7.522497445 milliseconds, satellite range: 1127.593999667635 kilometers

Transmit time: 2018-07-12 00:29:03.897022, receive time: 2018-07-12 00:29:03.904656
Time of flight: 7.633188421 milliseconds, satellite range: 1144.1861595543644 kilometers

Transmit time: 2018-07-12 00:29:26.906102, receive time: 2018-07-12 00:29:26.914052
Time of flight: 7.949608287 milliseconds,

Transmit time: 2018-07-13 15:27:09.500606, receive time: 2018-07-13 15:27:09.511380
Time of flight: 10.773991531 milliseconds, satellite range: 1614.9807017748365 kilometers

Transmit time: 2018-07-13 15:28:24.800609, receive time: 2018-07-13 15:28:24.808793
Time of flight: 8.183542214000001 milliseconds, satellite range: 1226.682117740911 kilometers

Transmit time: 2018-07-13 15:30:41.400606, receive time: 2018-07-13 15:30:41.407336
Time of flight: 6.730648708 milliseconds, satellite range: 1008.8988600529221 kilometers

Transmit time: 2018-07-13 21:35:27.508564, receive time: 2018-07-13 21:35:27.517478
Time of flight: 8.914071904 milliseconds, satellite range: 1336.1857634444502 kilometers

Transmit time: 2018-07-13 21:35:37.143064, receive time: 2018-07-13 21:35:37.151774
Time of flight: 8.709015166 milliseconds, satellite range: 1305.4485316872092 kilometers

Transmit time: 2018-07-13 21:35:52.816264, receive time: 2018-07-13 21:35:52.824684
Time of flight: 8.419260445 milliseconds